<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

In [31]:
import pandas as pd  
import numpy as np  
import os  
import sys  
import collections
from itertools import chain
import json
from IPython.display import display, HTML
import csv
import glob
import h5py
import re


In [32]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)
pd.options.display.float_format = lambda x: '{:,.2f}'.format(x) if x > 1 else '{:,.2%}'.format(x)

In [33]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [34]:
top_metrics_file = 'top_metrics_for_tbl.csv'
input_h5_file = ''
input_base_file_name = ''

In [35]:
# Parameters
top_metrics_file = (
    "/home/gat.krieger/proj/VariantCalling/ugvc/reports/top_metrics_for_tbl.csv"
)
input_h5_file = "/data/report_fixes/ssqc_from_sorter/026532-Lb_1866-Z0058-CATCTCAGTGCAATGAT.aggregated_metrics.h5"
input_base_file_name = "026532-Lb_1866-Z0058-CATCTCAGTGCAATGAT"


In [36]:
if not os.path.isfile(top_metrics_file):
    raise ValueError(f"Input {top_metrics_file} does not exist")


In [37]:
   
dict_features = {row[0] : row[1] for _, row in pd.read_csv(top_metrics_file).iterrows()}
df_features = pd.read_csv(top_metrics_file)
list_metrics = list(set(df_features['metric'] ))

In [38]:
# get Keys within the H5 file
f = h5py.File(input_h5_file, 'r')
list_keys = list(f.keys())
hist_list_keys =  [i for i in list_keys if i.startswith('histogram_')]
tbl_list_keys = pd.DataFrame(list(set(list_keys) - set(hist_list_keys)))
tbl_list_keys.columns = ['metric']
del(list_keys)

# create table merging top required metrics to display and input provided
# ....................
tbl_top_values = pd.merge(df_features,tbl_list_keys,on='metric',how='inner')

In [39]:

HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Input parameters"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")



In [40]:
to_print_sample_info = pd.DataFrame(
        data = {
            'value' : [input_base_file_name, str(input_h5_file)]
        },
        index = ['Sample name', 'h5 file'])

to_print_sample_info['value'] = to_print_sample_info['value'].str.wrap(100)
def wrap_df_text(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))

wrap_df_text(to_print_sample_info.style.set_properties(**{'text-align': 'left'}))

,value
Sample name,026532-Lb_1866-Z0058-CATCTCAGTGCAATGAT
h5 file,/data/report_fixes/ssqc_from_sorter/026532-Lb_1866-Z0058-CATCTCAGTGCAATGAT.aggregated_metrics.h5


In [41]:
HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Summary View: Main Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [42]:
separator = '___'
i_value= np.nan 
to_print = pd.DataFrame()

tbl_top = pd.DataFrame()

In [43]:

for temp_metric in tbl_top_values['metric'].unique():
    list_top_tbl = sub_top_tbl = pd.DataFrame()
    sub_top_tbl =  tbl_top_values[tbl_top_values['metric'] == temp_metric]
    df_h5_tbl = pd.read_hdf(input_h5_file, temp_metric).T
    # display(df_h5_tbl.index)
    df_h5_tbl = df_h5_tbl.reset_index()
    
    # stats_coverage is a multiindex dataframe    
    if (temp_metric.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        df_h5_tbl = df_h5_tbl[['key', 'value']]        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = pd.concat((to_print,list_top_tbl))
        
    else:
        df_h5_tbl.columns = ['key','value']        
        list_top_tbl = pd.merge(df_h5_tbl, sub_top_tbl ,on='key',how='inner')
        to_print = pd.concat((to_print, list_top_tbl))
# display(to_print)
# if type(to_print.loc[to_print.key == 'PCT_PF_READS', 'value'].values[0]) is int:
#     to_print.loc[to_print.key == 'PCT_PF_READS', 'value'] = '100%'

to_print.index = to_print['key']
display(to_print['value'].to_frame())


,value
key,
PF_MISMATCH_RATE,38.00%
PF_INDEL_RATE,44.00%
MEAN_READ_LENGTH,260.07
MEAN_ALIGNED_READ_LENGTH,260.39
PF_BASES,"31,777,881,341.00"
MEAN_COVERAGE,9.98
PCT_20X,1.03
Genome___percentile_5,3.81
Simple repeat___percentile_5,1.62%


In [46]:
tbl_top_values

,key,metric
0,TOTAL_READS,AlignmentSummaryMetrics
1,PCT_PF_READS,AlignmentSummaryMetrics
2,PCT_PF_READS_ALIGNED,AlignmentSummaryMetrics
3,MEAN_READ_LENGTH,AlignmentSummaryMetrics
4,MEAN_ALIGNED_READ_LENGTH,AlignmentSummaryMetrics
5,PF_INDEL_RATE,AlignmentSummaryMetrics
6,PF_MISMATCH_RATE,AlignmentSummaryMetrics
7,PF_BASES,QualityYieldMetricsFlow
8,PF_Q30_BASES,QualityYieldMetricsFlow
9,MEAN_COVERAGE,RawWgsMetrics


In [44]:
HTML("<b>"+""+"</b>")
HTML("<hr/>")
HTML("<h2 style=\"font-size:20px;\">"+"Detailed View: All Metrics"+"</h2>")
HTML("<hr/>")
HTML("<b>"+""+"</b>")


In [45]:
to_print = pd.DataFrame()
sorted_keys  = tbl_list_keys['metric'].sort_values()

for tbl_key in sorted_keys:
    
    HTML("<br>"+"<br>"+"<b>"+"Metric type: "+tbl_key+"</b>"+"<br>")
    df_h5_tbl = pd.read_hdf(input_h5_file, tbl_key).T
    df_h5_tbl = df_h5_tbl.reset_index()
    

    if (tbl_key.startswith('stats') ):
        df_h5_tbl['metric'] = df_h5_tbl['level_0'] + separator + df_h5_tbl['level_1']
        df_h5_tbl = df_h5_tbl.drop(columns=['level_0','level_1']).copy()
        df_h5_tbl.columns = ['value','key']
        to_print = df_h5_tbl[['key', 'value']]        

    else:
        to_print = df_h5_tbl
        to_print.columns = ['key','value'] 
        
    to_print.index = to_print['key']
    display(to_print['value'].to_frame())
        


,value
key,
PCT_PF_ALIGNED,98.49
PCT_PF_HQ_ALIGNED,91.68
PF_MISMATCH_RATE,38.00%
PF_INDEL_RATE,44.00%
MEAN_READ_LENGTH,260.07
MEDIAN_READ_LENGTH,284.00
MEAN_ALIGNED_READ_LENGTH,260.39
MQ_GTE_1,"114,614,928.00"
MQ_GTE_10,"113,380,377.00"


,value
key,
PERCENT_DUPLICATION,7.72


,value
key,
PF_READS,"122,191,864.00"
PCT_PF_Q20_BASES,85.21
MEAN_QUALITY,36.63
NON_PF_READS,0.00%
PCT_NON_PF_READS,0.00%
TOTAL_READS,"122,191,864.00"
TOTAL_BASES,"31,777,881,341.00"
PF_BASES,"31,777,881,341.00"


,value
key,
MEAN_COVERAGE,9.98
MEDIAN_COVERAGE,10.00
PCT_1X,99.52
PCT_5X,94.52
PCT_10X,51.03
PCT_20X,1.03
PCT_50X,11.00%
PCT_100X,4.00%
PCT_500X,1.00%


,value
key,
Genome___percentile_5,3.81
Genome___percentile_10,5.00
Genome___median_coverage,9.09
Genome___median_coverage_normalized,100.00%
Genome___percent_larger_than_05_of_genome_median,94.52
Genome___percent_larger_than_025_of_genome_median,98.43
Genome___percent_over_or_equal_to_10x,51.03
Genome___percent_over_or_equal_to_20x,1.03
Exome___percentile_5,4.24
